# Collecting data

This notebook focuses on collecting data using Web Scraping techniques. So use this notebook to do Scraping.

**Import Library**

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
urls = ['https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1.html']
cases = []

In [3]:
for i in range(2, 6):
   urls.append(f"https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1/page/{i}.html")

print(urls)

['https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1.html', 'https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1/page/2.html', 'https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1/page/3.html', 'https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1/page/4.html', 'https://putusan3.mahkamahagung.go.id/direktori/index/kategori/pidana-umum-1/page/5.html']


Fungsi `request_page(url)` dan `request_detail_page(url)` digunakan untuk:

- **Mengambil konten halaman web:** Fungsi ini mengirimkan permintaan (request) ke URL yang diberikan.
- **Menganalisis konten:** Setelah mendapatkan konten halaman, fungsi ini akan memparsing HTML untuk menemukan elemen-elemen tertentu, khususnya yang berkaitan dengan URL yang ingin diekstrak.

In [ ]:
def request_page(url):
   response = requests.get(url)
   
   if response.status_code >= 400:
      return None
   
   soup = BeautifulSoup(response.content, 'html.parser')
   posts_list = soup.find('div', id='popular-post-list-sidebar')
   if posts_list is None:
      return None

   return posts_list.find_all('div', class_='spost clearfix')


def request_detail_page(url):
   response = requests.get(url)
   if response.status_code >= 400:
      return None

   soup = BeautifulSoup(response.content, 'html.parser')

   detail_case = soup.find('div', id='popular-post-list-sidebar')
   if detail_case is None:
      return None
   
   detail_case = detail_case.find('table', class_='table')
   if detail_case is None:
      return None
   
   return detail_case

`get_case` dirancang untuk mengekstrak informasi spesifik dari sebuah halaman HTML yang mewakili detail suatu kasus. Fungsi ini melakukan parsing HTML yang kompleks untuk menemukan dan mengembalikan bagian teks tertentu.


In [5]:
def get_case(detail_case):
   '''
      Argument:
      - `detail_case`: Sebuah objek BeautifulSoup yang merepresentasikan konten HTML dari halaman detail kasus.
   
      Return:
      - String
   '''

   tag_tr = detail_case.find_all('tr')[14]
   tag_td = tag_tr.find_all('td')[1]
   tag_ol = tag_td.find('ol')

   if tag_ol is None:
      return ""
   
   tag_li = tag_ol.find_all('li')[0]

   pattern_1 = r"\?(.*?)\?"
   pattern_2 = r"\'(.*?)\'"
   
   pattern_3 = r"\?(.*?)\'"
   pattern_4 = r"\'(.*?)\?"
   
   pattern_5 = r"\"(.*?)\?"
   pattern_6 = r"\?(.*?)\""

   result = re.search(pattern_1 + "|" +
                      pattern_2 + "|" +
                      pattern_3 + "|" +
                      pattern_4 + "|" +
                      pattern_5 + "|" +
                      pattern_6
                     , tag_li.text.strip())

   if result is None:
      return ""

   return result.group()[1:-1]


Pada sel ini, berguna untuk **mengekstrak URL-URL spesifik dari beberapa halaman web**. Prosesnya melibatkan langkah-langkah berikut:

1. **Mengambil Konten Halaman:** Untuk setiap URL dalam list `urls`, kode akan mengambil seluruh konten HTML halaman tersebut.
2. **Mencari Elemen yang Diinginkan:** Kode kemudian mencari elemen HTML `<strong>` yang biasanya menandai teks penting.
3. **Mengekstrak Tautan:** Di dalam setiap elemen `<strong>`, kode mencari tautan (link) yang ditandai dengan tag `<a>`.
4. **Menyimpan URL:** URL yang ditemukan kemudian disimpan dalam sebuah list bernama `cases`.

In [6]:
for url in urls:
   posts = request_page(url)

   for post in posts:
      strong_tags = post.find_all('strong')

      for strong_tag in strong_tags:
         a = strong_tag.find('a')
         if a:
            cases.append({ 'link' : a['href'] })

print(cases)
print(f"Total link detail page: {len(cases)}")

[{'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57c9a91a8ccafd5313431333235.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57baed4b9ec9914313430363530.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57ba5cef2549cfc313430363335.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57b0c04c14481ea313430323137.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57aa0bbddaa8ee1313335393137.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa579ec2e3c34b9a4313335343134.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa579a0398a868bdc313335323036.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57992145828b7bc313335313433.html'}, {'link': 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa5797a8c9922865f313335313033.html'}, {'link': 'https://putusan3.mahkamahagung.go.i

#### Fungsi `request_detail_page`

Fungsi `request_detail_page` digunakan untuk mengambil informasi detail tentang sebuah kasus dari halaman web. Fungsi ini menerima sebuah link sebagai input dan mengembalikan konten halaman tersebut dalam bentuk yang dapat diproses oleh BeautifulSoup.

In [ ]:
for index, case in enumerate(cases):
   print(f"index: {index}")
   print(f"link: {case['link']}")
   detail_case = request_detail_page(case['link'])
   if detail_case is None:
      case['kasus'] = ''
      continue
   
   case['kasus'] = get_case(detail_case).strip()
   case['pasal'] = ''

index: 0
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57c9a91a8ccafd5313431333235.html
index: 1
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57baed4b9ec9914313430363530.html
index: 2
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57ba5cef2549cfc313430363335.html
index: 3
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57b0c04c14481ea313430323137.html
index: 4
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57aa0bbddaa8ee1313335393137.html
index: 5
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa579ec2e3c34b9a4313335343134.html
index: 6
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa579a0398a868bdc313335323036.html
index: 7
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa57992145828b7bc313335313433.html
index: 8
link: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefa5797a8c9922865f313335313033.html
index: 9
link: https://putus

In [13]:
df = pd.DataFrame(cases)

df.head()

,link,kasus,pasal
0,https://putusan3.mahkamahagung.go.id/direktori...,Turut serta melakukan Penggelapan,
1,https://putusan3.mahkamahagung.go.id/direktori...,Pencurian Dengan Pemberatan,
2,https://putusan3.mahkamahagung.go.id/direktori...,Penggelapan,
3,https://putusan3.mahkamahagung.go.id/direktori...,,
4,https://putusan3.mahkamahagung.go.id/direktori...,secara bersama-sama melakukan Penggelapan Yang...,


Menyimpan sebuah DataFrame (yang direpresentasikan oleh variabel `df`) ke dalam sebuah file CSV.

In [14]:
df.to_csv('../data/csv/dataset_1.csv', index=False)